# Yarkovsky Effect

This example will demonstrate how to use both versions of the Reboundx Yarkovsky effect in a Rebound simulation. We will call these two versions the 'full version' and the 'max version.' A special parameter called 'yark_flag' is used to switch between the two. The difference between the versions and what situations they're better suited for is discussed in more detail below. 

For more information on this effect, please visit: (implementation paper in progress) 

We'll start with the full version.

## Full Version

This version of the effect is based off of the equations found in Veras et al. (2015a). A link to the paper is provided below. The full version can be used to get detailed calculations of the Yarkovsky effect on a particular body. However, it requires a large amount of parameters that may be difficult to find for a particular object. It also takes more computational time due to the large amount of equations that must be calaculated between each time step of the simulation. This version of the effect can be used to get accurate calculations on how a body is perturbed by the Yarkovsky effect.

Link to paper: https://ui.adsabs.harvard.edu/abs/2015MNRAS.451.2814V/abstract

Below is a simple example to show how to add the effect to a simulation. First, we create a Rebound simulation with the Sun and a test particle (which will be considered an asteroid) at .5 AU.

In [34]:
import rebound
import reboundx
import numpy as np
import time

#Simulation begins here
sim = rebound.Simulation()

sp = sim.particles #simplifies way to access particles parameters 

sim.units = ('yr', 'AU', 'Msun') #changes simulation and G to units of solar masses, years, and AU  
sim.integrator = "whfast" #integrator for sim
sim.dt = .05 #timestep for sim

sim.add(m=1) #Adds Sun 
sim.add(a=.5, f=0, Omega=0, omega=0, e=0, inc=0, m=0) #adds test particle 

#Moves all particles to center of momentum frame
sim.move_to_com()

#Gives orbital information before the simulation begins
print("\n***INITIAL ORBITS:***")
for orbit in sim.calculate_orbits():
    print(orbit)


***INITIAL ORBITS:***
<rebound.Orbit instance, a=0.5000000000000001 e=1.799893761170345e-16 inc=0.0 Omega=0.0 omega=0.0 f=0.0>


We then add the Yarkovsky effect from Reboundx and the required parameters for this version. Importantly, we must set 'yark_flag' to 0 to get the full version. As is standard for all Reboundx effects, the parameters must be inputed with the same units as the simulation (in this case it's AU/Msun/yr).

In [35]:
#Loads the effect into Rebound
rebx = reboundx.Extras(sim)
yark = rebx.load_force("yarkovsky_effect")

#These will be used to change the units of the parameters
au_conv = 1.495978707e11
msun_conv = 1.9885e30
yr_conv = 31557600.0

#Converts units of parameters from m/kg/sec to AU/Msun/yr
density = (3000.0*au_conv**3)/msun_conv #density of the object
c = (2.998e8*yr_conv)/au_conv #speed of light
lstar = (3.828e26*yr_conv**3)/(msun_conv*au_conv**2) #luminosity of star
radius = 1000/au_conv #radius of object
albedo = .017 #albedo of object
stef_boltz = ((5.670e-8)*yr_conv**3)/(msun_conv) #Stefan-Boltzmann constant
emissivity = .9 #emissivity of object
k = .25 #constant between
C = (680*yr_conv**2)/(au_conv**2) #specific heat capacity of object
K = (.04*yr_conv**3)/(au_conv*msun_conv) #thermal conductivity of object
rotation_period = 15470.9/yr_conv #rotation period of object

#These three parametrs decsribe the spin axis vector (or tilt) of the object
sx = 0
sy = 0
sz = 1 #for now we'll assume the object has no tilt and has a spin axis perpendicular to the orbital plane 

#Sets the parameters for the effect
sp[1].params["yark_flag"] = 0 #setting this flag to 0 will give us the full version of the effect
sp[1].params["body_density"] = density
yark.params["yark_c"] = c #set on the sim and not a particular particle
yark.params["lstar"] = lstar #set on the sim and not a particular particle
sp[1].r = radius #remember radius is not inputed as a Rebx parameter - it's inputed on the particle in the Rebound sim
sp[1].params["albedo"] = albedo
yark.params["stef_boltz"] = stef_boltz #set on the sim and not a particular particle
sp[1].params["emissivity"] = emissivity
sp[1].params["k"] = k
sp[1].params["specific_heat_capacity"] = C
sp[1].params["thermal_conductivity"] = K
sp[1].params["rotation_period"] = rotation_period
sp[1].params["spin_axis_x"] = sx
sp[1].params["spin_axis_y"] = sy
sp[1].params["spin_axis_z"] = sz

rebx.add_force(yark) #adds the force to the simulation

We integrate this system for 100,000 years and print out the difference between the particle's semi-major axis before and after the simulation.

In [36]:
tmax=100000 # in yrs

a_start = .5 #starting semi-major axis for the asteroid
   
timer_start = time.perf_counter() #starts timer for sim
    
sim.integrate(tmax) #integrates system for tmax years  

timer_end = time.perf_counter() #stops timer for sim
 
a_final = sp[1].a #semi-major axis of asteroid after the sim    
                      
print("CHANGE IN SEMI-MAJOR AXIS:", a_final-a_start, "AU\n") #prints difference between the intitial and final semi-major axes of asteroid

print("SIMULATION RUNTIME:", timer_end-timer_start, "sec") #prints out difference between the start and end of the timer

CHANGE IN SEMI-MAJOR AXIS: 1.814253269349919e-05 AU

SIMULATION RUNTIME: 3.327513808995718 sec


## Max Version

This version of the effect is based off of equations from Veras et al. (2019). Once again, a link to this paper is provided below. This version artificially places values in a rotation matrix from the original equations so that the output of the effect for a certain magnitude of accelertion is maximized. It requires less parameters than the full version and takes less computational time. However, it is mostly useful only to get upper bounds on how far the effect can push bodies inwards or outwards. This version of the effect is better for simulating a large groups of asteroids or trying to see general trends in the behavior of a body. 

Link to paper: https://ui.adsabs.harvard.edu/abs/2019MNRAS.485..708V/abstract

We'll use the same setup as before, but we'll also add another asteroid at .75 AU with identical physical properties. Let's start by creating a Rebound simulation again.

In [37]:
sim = rebound.Simulation()

sp = sim.particles #simplifies way to access particles parameters 

sim.units = ('yr', 'AU', 'Msun') #changes simulation and G to units of solar masses, years, and AU  
sim.integrator = "whfast" #integrator for sim
sim.dt = .05 #timestep for sim

sim.add(m=1) #Adds Sun 
sim.add(a=.5, f=0, Omega=0, omega=0, e=0, inc=0, m=0) #adds test particle 
sim.add(a=.75, f=0, Omega=0, omega=0, e=0, inc=0, m=0) #adds a second test particle

#Moves all particles to center of momentum frame
sim.move_to_com()

#Gives orbital information before the simulation begins
print("\n***INITIAL ORBITS:***")
for orbit in sim.calculate_orbits():
    print(orbit)


***INITIAL ORBITS:***
<rebound.Orbit instance, a=0.5000000000000001 e=1.799893761170345e-16 inc=0.0 Omega=0.0 omega=0.0 f=0.0>
<rebound.Orbit instance, a=0.75 e=0.0 inc=0.0 Omega=0.0 omega=0.0 f=0.0>


We then add the Yarkovsky effect from Reboundx and the necesary parameters for this version. This time, we must make sure that 'yark_flag' is set to 1 or -1 to get the max version of the effect. Setting it to 1 will push the asteroid outwards, while setting it to -1 will push it inwards. We'll push out our original asteroid and push in our new one. 

In [38]:
#Loads the effect into Rebound
rebx = reboundx.Extras(sim)
yark = rebx.load_force("yarkovsky_effect")

#These will be used to change the units of the parameters
au_conv = 1.495978707e11
msun_conv = 1.9885e30
yr_conv = 31557600.0

#Converts units of parameters from m/kg/sec to AU/Msun/yr
density = (3000*au_conv**3)/msun_conv #density of the object
c = (2.998e8*yr_conv)/au_conv #speed of light
lstar = (3.828e26*yr_conv**3)/(msun_conv*au_conv**2) #luminosity of star
radius = 1000/au_conv #radius of object



#Sets the parameters for the effect
yark.params["yark_c"] = c #set on the sim and not a particular particle
yark.params["lstar"] = lstar #set on the sim and not a particular particle

sp[1].params["yark_flag"] = 1 #setting this flag to 1 will give us the max outward version of the effect 
sp[1].params["body_density"] = density
sp[1].r = radius #remember radius is not inputed as a Rebx parameter - it's inputed on the particle in the Rebound sim 

sp[2].params["yark_flag"] = -1 #setting this flag to -1 will give us the max inward version of the effect 
sp[2].params["body_density"] = density
sp[2].r = radius #remember radius is not inputed as a Rebx parameter - it's inputed on the particle in the Rebound sim


rebx.add_force(yark) #adds the force to the simulation

Now we run the sim for 100,000 years and print out the results for both asteroids. As expected, the max version pushes our original asteroid farther out than the full version. Also note the difference in simulation times between the versions. Even with an extra particle, the max version was faster than the full version.

In [39]:
tmax=100000 # in yrs

a_start_1 = .5 #starting semi-major axis for the 1st asteroid
a_start_2 = .75 #starting semi-major axis for the 2nd asteroid

timer_start = time.perf_counter() #starts timer for sim

sim.integrate(tmax) #integrates system for tmax years    

timer_end = time.perf_counter() #stops timer for sim
 
a_final_1 = sp[1].a #semi-major axis of 1st asteroid after the sim
a_final_2 = sp[2].a #semi-major axis of 2nd asteroid after the sim
                      
print("CHANGE IN SEMI-MAJOR AXIS(Asteroid 1):", a_final_1-a_start_1, "AU\n")
print("CHANGE IN SEMI-MAJOR AXIS(Asteroid 2):", a_final_2-a_start_2, "AU\n")
print("SIMULATION RUNTIME:", timer_end-timer_start, "sec") #prints out difference between the start and end of the timer

CHANGE IN SEMI-MAJOR AXIS(Asteroid 1): 4.251832339163819e-05 AU

CHANGE IN SEMI-MAJOR AXIS(Asteroid 2): -3.4717212015644705e-05 AU

SIMULATION RUNTIME: 2.6195360510027967 sec
